# COMP9033 - Data Analytics Lab 06: Linear regression
## Introduction

This week's lab focuses on data modelling using linear regression. At the end of the lab, you should be able to use `scikit-learn` to:

- Create a linear regression model using the least squares technique.
- Use the model to predict new values.
- Measure the accuracy of the model.
- Engineer new features to optimise model performance.

### Getting started

Let's start by importing the packages we need. This week, we're going to use the `linear_model` subpackage from `scikit-learn` to build linear regression models using the least squares technique. We're also going to need the `dummy` subpackage to create a baseline regression model, to which we can compare.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict

Next, let's load the data. This week, we're going to load the [Auto MPG](https://archive.ics.uci.edu/ml/datasets/Auto+MPG) data set, which is available online at the [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). The dataset is in [fixed width format](https://www.ibm.com/support/knowledgecenter/SSFTDH_8.0.0/com.ibm.wbpm.wid.integ.doc/topics/rfixwidth.html), but fortunately this is supported out of the box by `pandas`' [`read_fwf`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_fwf.html) function:

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'

df = pd.read_fwf(url, header=None, names=['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
                                          'acceleration', 'model year', 'origin', 'car name'])

## Exploratory data analysis

According to its [documentation](https://archive.ics.uci.edu/ml/datasets/Auto+MPG), the Auto MPG dataset consists of eight explantory variables (i.e. features), each describing a single car model, which are related to the given target variable: the number of miles per gallon (MPG) of fuel of the given car. The following attribute information is given:

1. mpg: continuous
2. cylinders: multi-valued discrete
3. displacement: continuous
4. horsepower: continuous
5. weight: continuous
6. acceleration: continuous
7. model year: multi-valued discrete
8. origin: multi-valued discrete
9. car name: string (unique for each instance)

Let's start by taking a quick peek at the data:

In [ ]:
df.head()

As the car name is unique for each instance (according to the [dataset documentation](https://archive.ics.uci.edu/ml/datasets/Auto+MPG)), it cannot be used to predict the MPG by itself so let's drop it as a feature and use it as the index instead:

> **Note:** It seems plausible that MPG efficiency might vary from manufacturer to manufacturer, so we could generate a new feature by converting the car names into manufacturer names, but for simplicity lets just drop them here.

In [ ]:
df = df.set_index('car name')

df.head()

According to the [documentation](https://archive.ics.uci.edu/ml/datasets/Auto+MPG), the horsepower column contains a small number of missing values, each of which is denoted by the string `'?'`. Again, for simplicity, let's just drop these from the data set:

In [ ]:
df = df[df['horsepower'] != '?']

Usually, pandas is smart enough to recognise that a column is numeric and will convert it to the appropriate data type automatically. However, in this case, because there were strings present initially, the value type of the horsepower column isn't numeric:

In [ ]:
df.dtypes

We can correct this by converting the column values numbers manually, using `pandas`' [`to_numeric`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html) function:

In [ ]:
df['horsepower'] = pd.to_numeric(df['horsepower'])

# Check the data types again
df.dtypes

As can be seen, the data type of the horsepower column is now `float64`, i.e. a 64 bit floating point value.

According to the [documentation](https://archive.ics.uci.edu/ml/datasets/Auto+MPG), the origin variable is categoric (i.e. origin = 1 is not "less than" origin = 2) and so we should encode it via one hot encoding so that our model can make sense of it. This is easy with `pandas`: all we need to do is use the [`get_dummies`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) method, as follows:

In [ ]:
df = pd.get_dummies(df, columns=['origin'])

df.head()

As can be seen, one hot encoding converts the origin column into three separate binary columns, each representing the presence or absence of the given category.

Next, let's take a look at the distribution of the variables in the data frame. We can start by computing some descriptive statistics:

In [ ]:
df.describe()

Print a matrix of pairwise Pearson correlation values:

In [ ]:
df.corr()

Let's also create a scatter plot matrix:

In [ ]:
pd.plotting.scatter_matrix(df, s=50, hist_kwds={'bins': 10}, figsize=(16, 16));

Based on the above information, we can conclude the following:

- Based on a quick visual inspection, there don't appear to be significant numbers of outliers in the data set. (We could make boxplots for each variable - but let's save time and skip it here.)
- Most of the explanatory variables appear to have a non-linear relationship with the target.
- There is a high degree of correlation ($r > 0.9$) between cylinders and displacement and, also, between weight and displacement.
- The following variables appear to be left-skewed: mpg, displacement, horsepower, weight.
- The acceleration variable appears to be normally distributed.
- The model year follows a rough uniform distributed.
- The cylinders and origin variables have few unique values.

For now, we'll just note this information, but we'll come back to it later when improving our model.

## Data Modelling
### Dummy model

Let's start our analysis by building a dummy regression model that makes very naive (often incorrect) predictions about the target variable. This is a good first step as it gives us a benchmark to compare our later models to.

Creating a dummy regression model with `scikit-learn` is easy: first, we create an instance of [`DummyRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyRegressor.html#sklearn.dummy.DummyRegressor), and then we evaluate its performance on the data using cross validation, just like last week.

> **Note:** Our dummy model has no hyperparameters, so we don't need to do an inner cross validation or grid search - just the outer cross validation to estimate the model accuracy.

In [ ]:
X = df.drop('mpg', axis='columns')  # X = features
y = df['mpg']                       # y = prediction target

model = DummyRegressor()  # Predicts the target as the average of the features

outer_cv = KFold(n_splits=5, shuffle=True, random_state=0)  # 5 fold cross validation
y_pred = cross_val_predict(model, X, y, cv=outer_cv)

print('Mean absolute error: %f' % mean_absolute_error(y, y_pred))
print('Standard deviation of the error: %f' % (y - y_pred).std())

ax = (y - y_pred).hist()
ax.set(
    title='Distribution of errors for the dummy model',
    xlabel='Error'
);

The dummy model predicts the MPG with an average error of approximately $\pm 6.57$ (although, as can be seen from the distribution of errors the spread is much larger than this). Let's see if we can do better with a linear regression model.

### Linear regression model

`scikit-learn` supports linear regression via its [`linear_model`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) subpackage. This subpackage supports least squares regression, lasso regression and ridge regression, as well as many other varieties. Let's use least squares to build our model. We can do this using the [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) class, as follows:

In [ ]:
X = df.drop('mpg', axis='columns')
y = df['mpg']

model = LinearRegression()

outer_cv = KFold(n_splits=5, shuffle=True, random_state=0)
y_pred = cross_val_predict(model, X, y, cv=outer_cv)

print('Mean absolute error: %f' % mean_absolute_error(y, y_pred))
print('Standard deviation of the error: %f' % (y - y_pred).std())

ax = (y - y_pred).hist()
ax.set(
    title='Distribution of errors for the linear regression model',
    xlabel='Error'
);

Our linear regression model predicts the MPG with an average error of approximately $\pm 2.59$ and a significantly smaller standard deviation too - this is a big improvement over the dummy model!

But we can do better! Earlier, we noted that several of the features had non-linear relationships with the target variable - if we could transform these variables, we might be able to make this relationship more linear. Let's consider the displacement, horsepower and weight variables:

In [ ]:
pd.plotting.scatter_matrix(df[['mpg', 'displacement', 'horsepower', 'weight']], s=50, figsize=(9, 9));

The relationship between the target and these predictors appears to be an exponentially decreasing one: as the predictors increase in value, there is an exponential decrease in the target value. Log-transforming the variables should help to remove this effect (logarithms are the inverse mathematical operation to exponentials):

In [ ]:
df['displacement'] = df['displacement'].map(np.log)
df['horsepower'] = df['horsepower'].map(np.log)
df['weight'] = df['weight'].map(np.log)

Now, the relationship between the predictors and the target is much more linear:

In [ ]:
pd.plotting.scatter_matrix(df[['mpg', 'displacement', 'horsepower', 'weight']], s=50, figsize=(9, 9));

Let's run the analysis a second time and see the effect this has had:

In [ ]:
X = df.drop('mpg', axis='columns')
y = df['mpg']

model = LinearRegression()

outer_cv = KFold(n_splits=5, shuffle=True, random_state=0)
y_pred = cross_val_predict(model, X, y, cv=outer_cv)

print('Mean absolute error: %f' % mean_absolute_error(y, y_pred))
print('Standard deviation of the error: %f' % (y - y_pred).std())

ax = (y - y_pred).hist()
ax.set(
    title='Distribution of errors for the linear regression model with transformed features',
    xlabel='Error'
);

As can be seen, the average error has now decreased to $\pm 2.33$ and the standard deviation of the error to 3.12. Further reductions in error might be achieved by experimenting with feature selection, given the high degree of correlation between some of the predictors, or with a more sophisticated model, such as ridge regression.